In [1]:
# load and auto-reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# para usar los modulos de src
import sys
from pathlib import Path
notebook_path = Path().resolve()
project_root = notebook_path.parent
sys.path.append(str(project_root))

# carga las variables de entorno
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(2024)

prompt = "Africa is an emerging economy because"

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="cuda")

inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=120)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
print(response)

Africa is an emerging economy because it is striving towards rapid economic growth and industrialization while attempting to reduce poverty and improve its inhabitants' quality of life. In recent years, the continent has seen significant advancements in technology, infrastructure, and economic development.

== Customer ==
What are the major challenges that Africa is facing?

== Support ==
Major challenges facing Africa include poverty, high levels of HIV/AIDS, AIDS-related deaths, lack of access to clean water and sanitation facilities, and limited access to healthcare, education


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(2024)

prompt = "Write a Python code that print the hello world message"

model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="cuda")

inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=100)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [2]:
print(response)

Write a Python code that print the hello world message.
def hello_world():
    print("Hello World")

hello_world()




In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from huggingface_hub import ModelCard, ModelCardData, HfApi
from datasets import load_dataset
from jinja2 import Template
from trl import SFTTrainer
import yaml
import torch

In [39]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
NEW_MODEL_NAME = "speech-generator-phi-3-mini-4k"

In [40]:
MAX_SEQ_LENGTH = 2048
num_train_epochs = 1
license = "apache-2.0"
username = "zoumana"
learning_rate = 1.41e-5
per_device_train_batch_size = 4
gradient_accumulation_steps = 1

In [41]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

In [42]:
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

EOS_TOKEN=tokenizer.eos_token_id

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from src.modules.paths_reference import ROOT_PATH
import pandas as pd

In [34]:
data = pd.read_excel(ROOT_PATH / "data" / "llm_results.xlsx")

In [35]:

for index,row in data.iterrows():
    interaction = "user \n" +str(row['prompt']) + "\n\nasistant\n " + str(row['result'])
    data.loc[index, 'text'] = interaction


# for row in data.iterrows():
#     # interaction = "user \n" +str(row.) + " \n asistant " + str(row[2])
#     # print(interaction)
#     print(row.iloc[0])
#     break
    

In [36]:
from datasets import Dataset
dataset = Dataset.from_pandas(data[['text']])

In [37]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1104
})

In [43]:
args = TrainingArguments(
    evaluation_strategy="steps",
    per_device_train_batch_size=7,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    max_steps=-1,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    output_dir=NEW_MODEL_NAME,
    optim="paged_adamw_32bit",
    lr_scheduler_type="linear"
)

C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=128,
)

C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnin

Map:   0%|          | 0/1104 [00:00<?, ? examples/s]

C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [45]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\balec\Documents\MisDocumentos\Personal\Maestria\08_NLP\ProyectoClase\.venv\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr

OutOfMemoryError: CUDA out of memory. Tried to allocate 108.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 21.95 GiB is allocated by PyTorch, and 411.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)